In [1]:
!pip install langchain
!pip install openai
!pip install PyPDF2
!pip install faiss-cpu
!pip install tiktoken
!pip install typing_extensions
!pip install -U langchain-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 811.8/811.8 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.4/239.4 kB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is n

In [2]:
from PyPDF2 import PdfReader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

In [21]:
OPENAI_API_KEY = "sk-liQwUlOylkko7Bt4IjQooP3BlBkFJaJbAzsf46h3y8t8DYyiP"

In [4]:
pdfreader = PdfReader("/content/Resume.pdf")

In [ ]:
raw_text = " "

pdfPages = pdfreader.pages
for i, page in enumerate(pdfPages):
  content = page.extract_text()
  if content:
    raw_text += content

# print(raw_text)

In [6]:
text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 800,
    chunk_overlap = 200,
    length_function = len
)

texts = text_splitter.split_text(raw_text)
# print(len(texts))

7


In [8]:
embeddings = OpenAIEmbeddings(openai_api_key = OPENAI_API_KEY)

In [9]:
document_search = FAISS.from_texts(texts, embeddings)

In [10]:
document_search

In [11]:
from langchain.chains.question_answering import load_qa_chain
from langchain_openai import OpenAI

In [18]:
chain = load_qa_chain(OpenAI(temperature=0.0,
                             openai_api_key = OPENAI_API_KEY),
                      chain_type = "stuff")

In [19]:
question_s = "Please, Tell me about yourself"
query_s = f"""You are an interview consultant. Your job is to provide the answer of an \
        interview question in 500 words, delimited by triple backticks. by analyzing the given resume.
        interview question:  ```{question_s}``` """

In [20]:
docs = document_search.similarity_search(query_s)
response = chain.invoke(input = {"input_documents": docs, "question": query_s})
response["output_text"]

'\nThe candidate, Dipankar Porey, has a strong background in technology consulting, data science, automation, and development. They have experience working at EY - Ernst & Young LLP in Mumbai, India, where they were responsible for monitoring project integration and developing an end-to-end data integration process using Sharepoint and GCP. They also have experience with automation, using tools like Streamlit and libraries such as OpenAI and HuggingFace to create a generative AI platform that can respond to user-specific questions.\n\nIn terms of skills, Dipankar has a strong proficiency in Python, C, C++, MySQL, SQL, Tableau, HTML, CSS, JavaScript, Flask, Streamlit, Power BI, and Git. They also have experience with cloud platforms like Azure, AWS, and GCP, as well as various software tools like MS Word, Excel, PowerPoint, VS Code, PyCharm, Jupyter, Anaconda, and Google Colab. In terms of AI and data science, they have experience with PyTorch, TensorFlow, Pandas, NumPy, Matplotlib, Sci